In [ ]:
import graphviz
import networkx

def deBruijn(reads:list, k:int, DisplayGraph=True):
    graphDict = {}
    
    for read in reads:        
        # Get K-mers
        Kmers = [] 
        for i in range(len(read)):
            if len(read[i:i+k])==k:
                Kmers.append(read[i:i+k])
        
        #Get L and R K-1-mers
        lkmer =[]
        rkmer =[]
        for Kmer in Kmers:
            lkmer.append(Kmer[0:len(Kmer)-1])
            rkmer.append(Kmer[1:len(Kmer)])
            
            graphDict.setdefault(lkmer[-1], [])
            graphDict[lkmer[-1]].append(rkmer[-1])
            
    #Remove Tips
    for key in graphDict.keys():
        lst = graphDict[key]
        simplifiedEdges = []
        if len(lst) <= 1:
            continue
        mostCommon = max(set(lst), key=lst.count)
        for val in graphDict[key]:
            if graphDict.get(val) is None:
                simplifiedEdges.append(mostCommon)
            else:
                simplifiedEdges.append(val)
        
        graphDict[key] = [mostCommon for i in lst]
        
    #Get path(s)
    G = nx.digraph.DiGraph()
    
    for key in graphDict.keys():
        for val in graphDict[key]:
            G.add_edge(key, val)
    if nx.is_eulerian(G):
        longestPath = eulerian_circuit(G)
    else:
        try:
            path = nx.dag_longest_path(G)
            longestPath = (path[0]+"".join([x[-1] for x in path[1:]]))
        except:
            longestPath = "Cycle in non-Eulerian de Bruijn Graph, sequence cannot be inferred"
        
    if DisplayGraph:
        #Rewrite multiple edges as Weights
        for key in graphDict.keys():
            weightedEdge = []
            for val in graphDict[key]:
                if (graphDict[key].count(val), val) in weightedEdge:
                    continue
                else:
                    weightedEdge.append((graphDict[key].count(val), val))
            graphDict[key] = weightedEdge
            
        #Create Graph
        deBruijnGraph = graphviz.Digraph(comment='de Bruijn Graph')
        #Create Nodes
        for key in graphDict.keys():
            deBruijnGraph.node(key,key)

        #Create Edges
        for key in graphDict.keys():
            for val in graphDict[key]:
                deBruijnGraph.edge(key, val[1], label=str(val[0]))

        display(deBruijnGraph)
    
    return longestPath